# Google Colabにて必要なモジュールを取得

In [1]:
!pip install langgraph langchain_openai langchain_core httpx requests

DEPRECATION: Loading egg at /Users/kunieda/.pyenv/versions/3.11.9/lib/python3.11/site-packages/tensorboardX-2.6.2.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/kunieda/.pyenv/versions/3.11.9/lib/python3.11/site-packages/six-1.16.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/kunieda/.pyenv/versions/3.11.9/lib/python3.11/site-packages/cycler-0.12.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/kunieda/.pyenv/versions/3

# LangChainで利用するLLMの設定

In [3]:
import os
from langchain_openai import AzureChatOpenAI

# 非推奨だが環境変数をベタがきしても良い
#os.environ["OPENAI_API_VERSION"] = "2024-xx-xx-preview"
#os.environ["AZURE_OPENAI_ENDPOINT"] = "https://xxxxxxxxxx.openai.azure.com"
#os.environ["AZURE_OPENAI_API_KEY"] = "xxxxxxxxxx"

# Google Colabで設定した場合
from google.colab import userdata
os.environ["OPENAI_API_VERSION"] = userdata.get("OPENAI_API_VERSION")
os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")


model = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    temperature=0,
)


# LangChain単体で利用して（Chainを使わずに）、簡単な小説を作成するコード

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

user_input = "ハッピーエンドのボーイミーツガール"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","ユーザが指定するジャンル等の情報をもとに、短編小説を生成するAIです。"),
        ("human", "{user_input}")
    ]
)

output_parser = StrOutputParser()

#LCELを使わない場合
prompt_value = prompt.invoke({"user_input": user_input})
ai_message = model.invoke(prompt_value)
output = output_parser.invoke(ai_message)

print(prompt_value)
print(ai_message)
print(output)

#テキストファイルに出力。Colabの場合は、Colabのフォルダからダウンロードする
with open("output_sample01.txt", "w") as f:
    f.write(output)

# 上記の単純な機能をLCELで書き直したコード

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

user_input = "ハッピーエンドのボーイミーツガール"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","ユーザが指定するジャンル等の情報をもとに、短編小説を生成するAIです。"),
        ("human", "{user_input}")
    ]
)

output_parser = StrOutputParser()

#ここでChainを組む
chain = prompt | model | output_parser

output = ""
#テキストが少しずつ生成されるようにするために、streamを使う
for chunk in chain.stream({"user_input":user_input}):
    print(chunk, end="", flush=True)
    output += chunk

#これまで通りinvokeを利用することもできる
#output = chain.invoke({"user_input":user_input})

with open("output_sample02.txt", "w") as f:
    f.write(output)

# プロットを生成してから小説を生成するように機能追加したコード

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

user_input = "ハッピーエンドのボーイミーツガール"

#CoTのプロンプトを定義
cot_system_prompt =  """
アナタはユーザが指定するジャンル等の情報をもとに、短編小説を生成するための情報を整理するAIです。
ユーザが指定したジャンルをもとに、短編小説を生成するための情報をステップバイステップで洗い出してください。

まずは、小説を書くうえで決めないといけない内容が何になるのかを、アナタ自身で全て洗い出してください。
その後、洗い出した項目を、アナタ自身が決定していってください。

ただし、作成する小説は、ユーザが指定したジャンルに沿った内容である必要があります。
しかしながら、山あり谷ありの非常に感動するようなストーリーを生成する必要があります。

また、使いたい名言や印象的なセリフなども書き出してください。

以上をもとに感動的なストーリーを生成するためのプロットをステップバイステップで書き出してください。
"""

#小説生成のプロンプトを定義
generate_story_system_prompt = """
アナタは、ユーザが指定するジャンル等の情報をもとに、短編小説を生成するです。
ユーザが指定したジャンルに加え、前段でLLMが洗い出した情報をもとに小説を作成してください。

前段のLLMが洗い出した情報を小説に入れ込む必要があるため、文字数は長くなっても構いません。
むしろ、前段の情報を洗い出していないのにも関わらず、ストーリーの生成を中断してはいけません。

またタイトルや段落などは表示しないでください。あくまで小説の本文のみを生成してください。

生成する文章はあくまでライトノベルのような内容である必要があります。
加えて、あらすじのような書き方は絶対にしてはいけません。
地の文で話を進めるのではなく、必ず、詳細な描写と登場人物間の会話でのやり取りをもとに、話を進めるようにしてください。

"""

generate_story_human_prompt = """
# ユーザが指定したジャンル等の情報
{user_input}

# 前段のLLMが洗い出した情報
{information}
"""

cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",cot_system_prompt),
        ("human", "{user_input}")
    ]
)

generate_story_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",generate_story_system_prompt),
        ("human", generate_story_human_prompt)
    ]
)

output_parser = StrOutputParser()

#CoTを用いて、プロットを生成するChainと、小説を生成するChainを繋げ、各出力を全て次のRunnableに渡す
chain = (
    RunnableParallel(
        {
            "user_input": RunnablePassthrough(),
            "information":cot_prompt|model|output_parser,
        }
    ).assign(novel=generate_story_prompt| model| output_parser)
)

output = ""
output_user_input = ""
output_information = ""
output_novel = ""
for chunk in chain.stream({"user_input":user_input}):
    if "user_input" in chunk:
        print(chunk["user_input"], end="", flush=True)
        output_user_input += chunk["user_input"]["user_input"]
    elif "information" in chunk:
        print(chunk["information"],end="", flush=True)
        output_information += chunk["information"]
    elif "novel" in chunk:
        print(chunk["novel"], end="", flush=True)
        output_novel += chunk["novel"]



with open("output_sample03.txt", "w") as f:
    f.write("\n====user_user_input====\n")
    f.write(output_user_input)
    f.write("\n")
    f.write("\n====output_information====\n")
    f.write(output_information)
    f.write("\n")
    f.write("\n====output_novel====\n")
    f.write(output_novel)
    f.write("\n")

# 別のAIエージェントにより改善案を出力してもらい、それを元に改善した小説を生成する機能を追加したコード

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

user_input = "ハッピーエンドのボーイミーツガール"

cot_system_prompt =  """
アナタはユーザが指定するジャンル等の情報をもとに、短編小説を生成するための情報を整理するAIです。
ユーザが指定したジャンルをもとに、短編小説を生成するための情報をステップバイステップで洗い出してください。

まずは、小説を書くうえで決めないといけない内容が何になるのかを、アナタ自身で全て洗い出してください。
その後、洗い出した項目を、アナタ自身が決定していってください。

ただし、作成する小説は、ユーザが指定したジャンルに沿った内容である必要があります。
しかしながら、山あり谷ありの非常に感動するようなストーリーを生成する必要があります。

また、使いたい名言や印象的なセリフなども書き出してください。

以上をもとに感動的なストーリーを生成するためのプロットをステップバイステップで書き出してください。
"""

generate_story_system_prompt = """
アナタは、ユーザが指定するジャンル等の情報をもとに、短編小説を生成するです。
ユーザが指定したジャンルに加え、前段でLLMが洗い出した情報をもとに小説を作成してください。

前段のLLMが洗い出した情報を小説に入れ込む必要があるため、文字数は長くなっても構いません。
むしろ、前段の情報を洗い出していないのにも関わらず、ストーリーの生成を中断してはいけません。

またタイトルや段落などは表示しないでください。あくまで小説の本文のみを生成してください。

生成する文章はあくまでライトノベルのような内容である必要があります。
加えて、あらすじのような書き方は絶対にしてはいけません。
地の文で話を進めるのではなく、必ず、詳細な描写と登場人物間の会話でのやり取りをもとに、話を進めるようにしてください。

"""

generate_story_human_prompt = """
# ユーザが指定したジャンル等の情報
{user_input}

# 前段のLLMが洗い出した情報
{information}
"""

criticalA_system_prompt = """
あなたは文芸出版社の編集者になり、私が添付ファイルとして送付する小説を読み、評価をしてください。

評価は、項目別の評点と、総合評価、およびコメントによって構成されます。

項目別の評点は、「オリジナリティー」「ストーリー」「設定」「キャラクター」「文章力」の5つの観点について、それぞれ良い順に、A、B+、B、B-、Cの5段階での評価をし、それらを踏まえた総合評価でも同様に、A、B+、B、B-、Cの5段階の評価を付けてください。

コメントでは、5つの観点の中で特筆すべき良い点と改善点について具体的に述べてください。

アナタは、編集者Aとして、小説の良かった点を特に重視して評価をして、最終選考に私の作品を推薦する立場から評価を出力してください。
"""

criticalB_system_prompt = """
あなたは文芸出版社の編集者になり、私が添付ファイルとして送付する小説を読み、評価をしてください。

評価は、項目別の評点と、総合評価、およびコメントによって構成されます。

項目別の評点は、「オリジナリティー」「ストーリー」「設定」「キャラクター」「文章力」の5つの観点について、それぞれ良い順に、A、B+、B、B-、Cの5段階での評価をし、それらを踏まえた総合評価でも同様に、A、B+、B、B-、Cの5段階の評価を付けてください。

コメントでは、5つの観点の中で改善点について具体的に述べてください。

アナタは、編集者Bとして、この小説には改善点が大きいという評価をして、厳しい評価をするとともに、最終選考に私の作品を推薦はしないという立場から評価を出力してください。
"""

taik_system_prompt = """
この小説をより良いものにするにはどうすれば良いかを二人の編集者が議論する対話を出力してください。すぐに結論を出さず、議論を様々な角度から深めてください。
"""

next_generation_prot_system_prompt = """
ここまでで、ユーザの指定したジャンルに沿った小説を生成し、編集者に評価をしてもらい、改善点を議論する対話を出力しました。
その情報をこれから提示するため、その内容に基づいて、元々の小説を改善するための情報をステップバイステップで洗い出してください。

作成する小説は、ユーザが指定したジャンルに沿った内容である必要があります。
しかしながら、山あり谷ありの非常に感動するようなストーリーを生成する必要があります。

また、使いたい名言や印象的なセリフなども書き出してください。

以上をもとに感動的なストーリーを生成するためのプロットをステップバイステップで書き出してください。

"""

next_generation_prot_human_prompt = """
# ユーザが指定したジャンル等の情報
{user_input}

# 過去に実際に記載した、改善が必要とユーザが判断した小説
{novel}

# 編集者からの評価
{critical_sheet}

# 編集者Aと編集者Bの議論
{taik}
"""


next_generation_novel_system_prompt = """
ここまでで、ユーザの指定したジャンルに沿った小説を生成し、編集者に評価をしてもらい、改善点を議論する対話を出力しました。
その上で、それまでの情報をもとに、改善した内容をLLMに生成してもらいました。

改善した内容と、過去の情報をもとに、実際に短編小説を生成してください。

前段のLLMが洗い出した情報を小説に入れ込む必要があるため、文字数は長くなっても構いません。
むしろ、前段の情報を洗い出していないのにも関わらず、ストーリーの生成を中断してはいけません。

生成する文章はあくまでライトノベルのような内容である必要があります。
加えて、あらすじのような書き方は絶対にしてはいけません。
地の文で話を進めるのではなく、必ず、詳細な描写と登場人物間の会話でのやり取りをもとに、話を進めるようにしてください。

山あり谷ありの感動的な話が生成されることを期待しています。アナタならできます。頑張ってください。
"""

next_generation_novel_human_prompt = """
# ユーザが指定したジャンル等の情報
{user_input}

# 過去に実際に記載した、改善が必要とユーザが判断した小説
{novel}

# 編集者からの評価
{critical_sheet}

# 編集者Aと編集者Bの議論
{taik}

# 上記の内容をもとにLLMが出した改善策
{next_novel_plot}
"""





cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",cot_system_prompt),
        ("human", "{user_input}")
    ]
)

generate_story_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",generate_story_system_prompt),
        ("human", generate_story_human_prompt)
    ]
)

criticalA_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",criticalA_system_prompt),
        ("human", "{novel}")
    ]
)

criticalB_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",criticalB_system_prompt),
        ("human", "{novel}")
    ]
)

talk_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",taik_system_prompt),
        ("human", "{critical_sheet}")
    ]
)

next_generation_prot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",next_generation_prot_system_prompt),
        ("human", next_generation_prot_human_prompt)
    ]
)

next_generation_novel_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",next_generation_novel_system_prompt),
        ("human", next_generation_novel_human_prompt)
    ]
)

output_parser = StrOutputParser()

chain = (
    RunnableParallel(
        {
            "user_input": RunnablePassthrough(),
            "information": cot_prompt | model | output_parser,
        }
    )
    .assign(novel=generate_story_prompt | model | output_parser)
    .assign(
        critical_sheet = RunnableParallel(
            {
            "criticalA_sheet":criticalA_prompt | model | output_parser,
            "criticalB_sheet":criticalB_prompt | model | output_parser
            }
        )
    )
    .assign(taik=talk_prompt | model | output_parser)
    .assign(next_novel_plot=next_generation_prot_prompt | model | output_parser)
    .assign(final_novel=next_generation_novel_prompt | model | output_parser)
)

output = ""
output_user_input = ""
output_information = ""
output_novel = ""
output_criticalA_sheet = ""
output_criticalB_sheet = ""
output_taik = ""
output_next_novel_plot = ""
output_final_novel = ""

for chunk in chain.stream({"user_input":user_input}):
    if "user_input" in chunk:
        print(chunk["user_input"], end="", flush=True)
        output_user_input += chunk["user_input"]["user_input"]
    elif "information" in chunk:
        print(chunk["information"],end="", flush=True)
        output_information += chunk["information"]
    elif "novel" in chunk:
        print(chunk["novel"], end="", flush=True)
        output_novel += chunk["novel"]
    elif "critical_sheet" in chunk:
        if "criticalA_sheet" in chunk["critical_sheet"]:
            print(chunk["critical_sheet"]["criticalA_sheet"], end="", flush=True)
            output_criticalA_sheet += chunk["critical_sheet"]["criticalA_sheet"]
        elif "criticalB_sheet" in chunk["critical_sheet"]:
            print(chunk["critical_sheet"]["criticalB_sheet"], end="", flush=True)
            output_criticalB_sheet += chunk["critical_sheet"]["criticalB_sheet"]
    elif "taik" in chunk:
        print(chunk["taik"], end="", flush=True)
        output_taik += chunk["taik"]
    elif "next_novel_plot" in chunk:
        print(chunk["next_novel_plot"], end="", flush=True)
        output_next_novel_plot += chunk["next_novel_plot"]
    elif "final_novel" in chunk:
        print(chunk["final_novel"], end="", flush=True)
        output_final_novel += chunk["final_novel"]




with open("output_sample04.txt", "w") as f:
    f.write("\n====user_user_input====\n")
    f.write(output_user_input)
    f.write("\n")
    f.write("\n====output_information====\n")
    f.write(output_information)
    f.write("\n")
    f.write("\n====output_novel====\n")
    f.write(output_novel)
    f.write("\n")
    f.write("\n====output_critical_sheet====\n")
    f.write("\n====output_criticalA_sheet====\n")
    f.write(output_criticalA_sheet)
    f.write("\n")
    f.write("\n====output_criticalB_sheet====\n")
    f.write(output_criticalB_sheet)
    f.write("\n")
    f.write("\n====output_taik====\n")
    f.write(output_taik)
    f.write("\n")
    f.write("\n====output_next_novel_plot====\n")
    f.write(output_next_novel_plot)
    f.write("\n")
    f.write("\n====output_final_novel====\n")
    f.write(output_final_novel)


# 小説家になろうAPIを利用して、トレンドを取得する機能を追加したコード

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
import requests

user_input = "ハッピーエンドのボーイミーツガール"

keyword_extraction_system_prompt = """
ユーザの入力から、大手小説投稿サイトにて、検索に利用するための重要なキーワード（複数）のみを抽出してください。
接続詞や助詞などは除外し、主要な名詞や形容詞だけを含めてください。
複数のキーワードがある場合は、必ず半角スペースで区切ってください。カンマなどでは区切らないでください。
ただし、抜き出すキーワードは最大3個までにしてください。重要度の高いキーワードから順に抽出してください。
"""

cot_system_prompt =  """
アナタはユーザが指定するジャンル等の情報をもとに、短編小説を生成するための情報を整理するAIです。
ユーザが指定したジャンルをもとに、短編小説を生成するための情報をステップバイステップで洗い出してください。

まずは、小説を書くうえで決めないといけない内容が何になるのかを、アナタ自身で全て洗い出してください。
その後、洗い出した項目を、アナタ自身が決定していってください。

ただし、作成する小説は、ユーザが指定したジャンルに沿った内容である必要があります。
しかしながら、山あり谷ありの非常に感動するようなストーリーを生成する必要があります。

また、使いたい名言や印象的なセリフなども書き出してください。

さらに、大規模小説投稿サイトにて高評価を獲得している、関連ジャンルの関連小説情報も参考にしてください。
例えば、関連小説情報の大まかなストーリーラインが今流行っている小説の概要です。なので、これから生成する小説もそのストーリーラインに揃えてください。

以上をもとに感動的なストーリーを生成するためのプロットをステップバイステップで書き出してください。
"""

cot_human_prompt = """
# ユーザが指定したジャンル等の情報
{user_input}

# 高評価を得ている取得したジャンルの関連小説情報
{naro_novel_info}
"""

generate_story_system_prompt = """
アナタは、ユーザが指定するジャンル等の情報をもとに、短編小説を生成するAIです。
ユーザが指定したジャンルに加え、前段でLLMが洗い出した情報をもとに小説を作成してください。

前段のLLMが洗い出した情報を小説に入れ込む必要があるため、文字数は長くなっても構いません。
むしろ、前段の情報を洗い出していないのにも関わらず、ストーリーの生成を中断してはいけません。

またタイトルや段落などは表示しないでください。あくまで小説の本文のみを生成してください。

生成する文章はあくまでライトノベルのような内容である必要があります。
加えて、あらすじのような書き方は絶対にしてはいけません。
地の文で話を進めるのではなく、必ず、詳細な描写と登場人物間の会話でのやり取りをもとに、話を進めるようにしてください。
"""

generate_story_human_prompt = """
# ユーザが指定したジャンル等の情報
{user_input}

# 前段のLLMが洗い出した情報
{information}
"""

criticalA_system_prompt = """
あなたは文芸出版社の編集者になり、私が添付ファイルとして送付する小説を読み、評価をしてください。

評価は、項目別の評点と、総合評価、およびコメントによって構成されます。

項目別の評点は、「オリジナリティー」「ストーリー」「設定」「キャラクター」「文章力」の5つの観点について、それぞれ良い順に、A、B+、B、B-、Cの5段階での評価をし、それらを踏まえた総合評価でも同様に、A、B+、B、B-、Cの5段階の評価を付けてください。

コメントでは、5つの観点の中で特筆すべき良い点と改善点について具体的に述べてください。

アナタは、編集者Aとして、小説の良かった点を特に重視して評価をして、最終選考に私の作品を推薦する立場から評価を出力してください。
"""

criticalB_system_prompt = """
あなたは文芸出版社の編集者になり、私が添付ファイルとして送付する小説を読み、評価をしてください。

評価は、項目別の評点と、総合評価、およびコメントによって構成されます。

項目別の評点は、「オリジナリティー」「ストーリー」「設定」「キャラクター」「文章力」の5つの観点について、それぞれ良い順に、A、B+、B、B-、Cの5段階での評価をし、それらを踏まえた総合評価でも同様に、A、B+、B、B-、Cの5段階の評価を付けてください。

コメントでは、5つの観点の中で改善点について具体的に述べてください。

アナタは、編集者Bとして、この小説には改善点が大きいという評価をして、厳しい評価をするとともに、最終選考に私の作品を推薦はしないという立場から評価を出力してください。
"""

taik_system_prompt = """
この小説をより良いものにするにはどうすれば良いかを二人の編集者が議論する対話を出力してください。すぐに結論を出さず、議論を様々な角度から深めてください。
"""

next_generation_prot_system_prompt = """
ここまでで、ユーザの指定したジャンルに沿った小説を生成し、編集者に評価をしてもらい、改善点を議論する対話を出力しました。
その情報をこれから提示するため、その内容に基づいて、元々の小説を改善するための情報をステップバイステップで洗い出してください。

作成する小説は、ユーザが指定したジャンルに沿った内容である必要があります。
しかしながら、山あり谷ありの非常に感動するようなストーリーを生成する必要があるため、そのためにどんなストーリーにしたらいいかをステップバイステップで概要を作成してください。

また、使いたい名言や印象的なセリフなども書き出してください。

"""

next_generation_prot_human_prompt = """
# ユーザが指定したジャンル等の情報
{user_input}

# 過去に実際に記載した、改善が必要とユーザが判断した小説
{novel}

# 編集者からの評価
{critical_sheet}

# 編集者Aと編集者Bの議論
{taik}
"""


next_generation_novel_system_prompt = """
ここまでで、ユーザの指定したジャンルに沿った小説を生成し、編集者に評価をしてもらい、改善点を議論する対話を出力しました。
その上で、それまでの情報をもとに、改善した内容をLLMに生成してもらいました。

改善した内容と、過去の情報をもとに、実際に短編小説を生成してください。

前段のLLMが洗い出した情報を小説に入れ込む必要があるため、文字数は長くなっても構いません。
むしろ、前段の情報を洗い出していないのにも関わらず、ストーリーの生成を中断してはいけません。

またタイトルや段落などは表示しないでください。あくまで小説の本文のみを生成してください。

生成する文章はあくまでライトノベルのような内容である必要があります。
加えて、あらすじのような書き方は絶対にしてはいけません。
地の文で話を進めるのではなく、必ず、詳細な描写と登場人物間の会話でのやり取りをもとに、話を進めるようにしてください。

山あり谷ありの感動的な話が生成されることを期待しています。アナタならできます。頑張ってください。
"""

next_generation_novel_human_prompt = """
# ユーザが指定したジャンル等の情報
{user_input}

# 過去に実際に記載した、改善が必要とユーザが判断した小説
{novel}

# 編集者からの評価
{critical_sheet}

# 編集者Aと編集者Bの議論
{taik}

# 上記の内容をもとにLLMが出した改善策
{next_novel_plot}
"""


# LangChainでキーワード抽出用プロンプトの設定
keyword_extraction_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", keyword_extraction_system_prompt),
        ("human", "{user_input}")
    ]
)


cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",cot_system_prompt),
        ("human", cot_human_prompt)
    ]
)

generate_story_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",generate_story_system_prompt),
        ("human", generate_story_human_prompt)
    ]
)

criticalA_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",criticalA_system_prompt),
        ("human", "{novel}")
    ]
)

criticalB_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",criticalB_system_prompt),
        ("human", "{novel}")
    ]
)

talk_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",taik_system_prompt),
        ("human", "{critical_sheet}")
    ]
)

next_generation_prot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",next_generation_prot_system_prompt),
        ("human", next_generation_prot_human_prompt)
    ]
)

next_generation_novel_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",next_generation_novel_system_prompt),
        ("human", next_generation_novel_human_prompt)
    ]
)

output_parser = StrOutputParser()

# なろうAPIを使って、ジャンルに基づいた評価の高い小説情報を取得する関数
def fetch_narou_novel_info(user_keyword):
    url = "https://api.syosetu.com/novelapi/api/"
    params = {
        "word": user_keyword,
        "order": "monthlypoint",
        "out": "json",
        "lim": 10
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        novels = response.json()[1:]
        return [{"title": novel.get("title", ""),
                 "summary": novel.get("story", ""),
                 "tag": novel.get("keyword", "").split()}
                for novel in novels]
    else:
        return ["検索した結果、関連小説情報を取得できませんでした。"]

chain = (
    RunnableParallel(
        {
            "user_input": RunnablePassthrough(),
            "search_words": keyword_extraction_prompt | model | output_parser,
        }
    )
    .assign(naro_novel_info=RunnableLambda(lambda inputs: {"naro_novel_info": fetch_narou_novel_info(inputs["search_words"])}))
    .assign(
        cot_prompt_output=RunnableLambda(lambda inputs: {
            "cot_prompt_output": cot_prompt.invoke({"user_input": inputs["user_input"], "naro_novel_info": inputs["naro_novel_info"]})
        })
    )
    .assign(information=cot_prompt | model | output_parser)
    .assign(novel=generate_story_prompt | model | output_parser)
    .assign(
        critical_sheet = RunnableParallel(
            {
            "criticalA_sheet":criticalA_prompt | model | output_parser,
            "criticalB_sheet":criticalB_prompt | model | output_parser
            }
        )
    )
    .assign(taik=talk_prompt | model | output_parser)
    .assign(next_novel_plot=next_generation_prot_prompt | model | output_parser)
    .assign(final_novel=next_generation_novel_prompt | model | output_parser)
)

output = ""
output_user_input = ""
output_information = ""
output_novel = ""
output_criticalA_sheet = ""
output_criticalB_sheet = ""
output_taik = ""
output_next_novel_plot = ""
output_final_novel = ""
output_naro_novel_info = ""
output_search_words = ""
output_cot_prompt_output = ""

for chunk in chain.stream({"user_input":user_input}):
    if "user_input" in chunk:
        print(chunk["user_input"]["user_input"], end="", flush=True)
        output_user_input += chunk["user_input"]["user_input"]
    elif "search_words" in chunk:
        print(chunk["search_words"], end="", flush=True)
        output_search_words += chunk["search_words"]
    elif "naro_novel_info" in chunk:
        print(chunk["naro_novel_info"], end="", flush=True)
        output_naro_novel_info += str(chunk["naro_novel_info"])
    elif "cot_prompt_output" in chunk:
        if "cot_prompt_output" in chunk:
            print(chunk["cot_prompt_output"]["cot_prompt_output"], end="", flush=True)
            output_cot_prompt_output += str(chunk["cot_prompt_output"]["cot_prompt_output"])
    elif "information" in chunk:
        print(chunk["information"],end="", flush=True)
        output_information += chunk["information"]
    elif "novel" in chunk:
        print(chunk["novel"], end="", flush=True)
        output_novel += chunk["novel"]
    elif "critical_sheet" in chunk:
        if "criticalA_sheet" in chunk["critical_sheet"]:
            print(chunk["critical_sheet"]["criticalA_sheet"], end="", flush=True)
            output_criticalA_sheet += chunk["critical_sheet"]["criticalA_sheet"]
        elif "criticalB_sheet" in chunk["critical_sheet"]:
            print(chunk["critical_sheet"]["criticalB_sheet"], end="", flush=True)
            output_criticalB_sheet += chunk["critical_sheet"]["criticalB_sheet"]
    elif "taik" in chunk:
        print(chunk["taik"], end="", flush=True)
        output_taik += chunk["taik"]
    elif "next_novel_plot" in chunk:
        print(chunk["next_novel_plot"], end="", flush=True)
        output_next_novel_plot += chunk["next_novel_plot"]
    elif "final_novel" in chunk:
        print(chunk["final_novel"], end="", flush=True)
        output_final_novel += chunk["final_novel"]




with open("output_sample05.txt", "w") as f:
    f.write("\n====user_user_input====\n")
    f.write(output_user_input)
    f.write("\n")
    f.write("\n====output_search_words====\n")
    f.write(output_search_words)
    f.write("\n")
    f.write("\n====output_naro_novel_info====\n")
    f.write(output_naro_novel_info)
    f.write("\n")
    f.write("\n====output_cot_prompt_output====\n")
    f.write(output_cot_prompt_output)
    f.write("\n")
    f.write("\n====output_information====\n")
    f.write(output_information)
    f.write("\n")
    f.write("\n====output_novel====\n")
    f.write(output_novel)
    f.write("\n")
    f.write("\n====output_critical_sheet====\n")
    f.write("\n====output_criticalA_sheet====\n")
    f.write(output_criticalA_sheet)
    f.write("\n")
    f.write("\n====output_criticalB_sheet====\n")
    f.write(output_criticalB_sheet)
    f.write("\n")
    f.write("\n====output_taik====\n")
    f.write(output_taik)
    f.write("\n")
    f.write("\n====output_next_novel_plot====\n")
    f.write(output_next_novel_plot)
    f.write("\n")
    f.write("\n====output_final_novel====\n")
    f.write(output_final_novel)


# より長編の小説を生成できるようにチャプターごとに小説を生成する機能を追加したコード



In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
import requests

user_input = "ハッピーエンドのボーイミーツガール"
all_chapter_number = 4

keyword_extraction_system_prompt = """
ユーザの入力から、大手小説投稿サイトにて、検索に利用するための重要なキーワード（複数）のみを抽出してください。
接続詞や助詞などは除外し、主要な名詞や形容詞だけを含めてください。
複数のキーワードがある場合は、必ず半角スペースで区切ってください。カンマなどでは区切らないでください。
ただし、抜き出すキーワードは最大3個までにしてください。重要度の高いキーワードから順に抽出してください。
"""

cot_system_prompt =  """
アナタはユーザが指定するジャンル等の情報をもとに、短編小説を生成するための情報を整理するAIです。
ユーザが指定したジャンルをもとに、長編小説を生成するための情報をステップバイステップで洗い出してください。

まずは、小説を書くうえで決めないといけない内容が何になるのかを、アナタ自身で全て洗い出してください。
その後、洗い出した項目を、アナタ自身が決定していってください。

ただし、作成する小説は、ユーザが指定したジャンルに沿った内容である必要があります。
しかしながら、山あり谷ありの非常に感動するようなストーリーを生成する必要があります。

また、使いたい名言や印象的なセリフなども書き出してください。

さらに、大規模小説投稿サイトにて高評価を獲得している、関連ジャンルの関連小説情報も参考にしてください。
例えば、関連小説情報の大まかなストーリーラインが今流行っている小説の概要です。なので、これから生成する小説もそのストーリーラインに揃えてください。

また、今回記載してもらう小説は、全{all_chapter_number}章により構成される小説です。
最後に、各章ごとにどんなストーリーにするかを、ユーザが指定したジャンルや、関連小説情報の概要をもとに、ステップバイステップで記載してください。

以上をもとに感動的なストーリーを生成するためのプロットをステップバイステップで書き出してください。
"""

cot_human_prompt = """
# ユーザが指定したジャンル等の情報
{user_input}

# 高評価を得ている取得したジャンルの関連小説情報
{naro_novel_info}
"""

generate_story_system_prompt = """
アナタは、ユーザが指定するジャンル等の情報をもとに、長編小説を生成するAIです。
ユーザが指定したジャンルに加え、前段でLLMが洗い出した情報をもとに小説を作成してください。

前段のLLMが洗い出した情報を小説に入れ込む必要があるため、文字数は長くなっても構いません。
むしろ、前段の情報を洗い出していないのにも関わらず、ストーリーの生成を中断してはいけません。

またタイトルや段落などは表示しないでください。あくまで小説の本文のみを生成してください。
また、今回記載してもらう小説は、全{all_chapter_number}章により構成される小説です。指定された章の小説を記載してください。

生成する文章はあくまでライトノベルのような内容である必要があります。
加えて、あらすじのような書き方は絶対にしてはいけません。
地の文で話を進めるのではなく、必ず、詳細な描写と登場人物間の会話でのやり取りをもとに、話を進めるようにしてください。
"""

generate_story_human_prompt = """

# 前段のLLMが考えた、今回書く小説のプロット情報
{information}

# 本小説の全話数
{all_chapter_number}

# 今回書いて欲しい話数
{chapter}

# 前話までの小説本文
{prev_novel}

----

この続きから記載してください

----


"""

criticalA_system_prompt = """
あなたは文芸出版社の編集者になり、私が添付ファイルとして送付する小説を読み、評価をしてください。

評価は、項目別の評点と、総合評価、およびコメントによって構成されます。

項目別の評点は、「オリジナリティー」「ストーリー」「設定」「キャラクター」「文章力」の5つの観点について、それぞれ良い順に、A、B+、B、B-、Cの5段階での評価をし、それらを踏まえた総合評価でも同様に、A、B+、B、B-、Cの5段階の評価を付けてください。

コメントでは、5つの観点の中で特筆すべき良い点と改善点について具体的に述べてください。

アナタは、編集者Aとして、小説の良かった点を特に重視して評価をして、最終選考に私の作品を推薦する立場から評価を出力してください。
"""

criticalB_system_prompt = """
あなたは文芸出版社の編集者になり、私が添付ファイルとして送付する小説を読み、評価をしてください。

評価は、項目別の評点と、総合評価、およびコメントによって構成されます。

項目別の評点は、「オリジナリティー」「ストーリー」「設定」「キャラクター」「文章力」の5つの観点について、それぞれ良い順に、A、B+、B、B-、Cの5段階での評価をし、それらを踏まえた総合評価でも同様に、A、B+、B、B-、Cの5段階の評価を付けてください。

コメントでは、5つの観点の中で改善点について具体的に述べてください。

アナタは、編集者Bとして、この小説には改善点が大きいという評価をして、厳しい評価をするとともに、最終選考に私の作品を推薦はしないという立場から評価を出力してください。
"""

taik_system_prompt = """
この小説をより良いものにするにはどうすれば良いかを二人の編集者が議論する対話を出力してください。すぐに結論を出さず、議論を様々な角度から深めてください。
"""

next_generation_prot_system_prompt = """
ここまでで、ユーザの指定したジャンルに沿った小説を生成し、編集者に評価をしてもらい、改善点を議論する対話を出力しました。
その情報をこれから提示するため、その内容に基づいて、元々の小説を改善するための情報をステップバイステップで洗い出してください。

作成する小説は、ユーザが指定したジャンルに沿った内容である必要があります。
しかしながら、山あり谷ありの非常に感動するようなストーリーを生成する必要があります。

また、使いたい名言や印象的なセリフなども書き出してください。

また、今回記載してもらう小説は、全{all_chapter_number}章により構成される小説です。
最後に、各章ごとにどんなストーリーにするかを、ステップバイステップで記載してください。

以上をもとに感動的なストーリーを生成するためのプロットをステップバイステップで書き出してください。

"""

next_generation_prot_human_prompt = """
# ユーザが指定したジャンル等の情報
{user_input}

# 過去に実際に記載した、改善が必要とユーザが判断した小説
{novel}

# 編集者からの評価
{critical_sheet}

# 編集者Aと編集者Bの議論
{taik}
"""


next_generation_novel_system_prompt = """
ここまでで、ユーザの指定したジャンルに沿った小説を生成し、編集者に評価をしてもらい、改善点を議論する対話を出力しました。
その上で、それまでの情報をもとに、改善した内容をLLMに生成してもらいました。

改善した内容と、過去の情報をもとに、実際に長編小説を生成してください。

前段のLLMが洗い出した情報を小説に入れ込む必要があるため、文字数は長くなっても構いません。
むしろ、前段の情報を洗い出していないのにも関わらず、ストーリーの生成を中断してはいけません。

またタイトルや段落などは表示しないでください。あくまで小説の本文のみを生成してください。
また、今回記載してもらう小説は、全{all_chapter_number}章により構成される小説です。指定された章の小説を記載してください。

生成する文章はあくまでライトノベルのような内容である必要があります。
加えて、あらすじのような書き方は絶対にしてはいけません。
地の文で話を進めるのではなく、必ず、詳細な描写と登場人物間の会話でのやり取りをもとに、話を進めるようにしてください。

山あり谷ありの感動的な話が生成されることを期待しています。アナタならできます。頑張ってください。
"""

next_generation_novel_human_prompt = """
# 過去に実際に記載した、改善が必要とユーザが判断した小説
{novel}

# 編集者からの評価
{critical_sheet}

# 編集者Aと編集者Bの議論
{taik}

# 上記の内容をもとにLLMが出した改善策
{next_novel_plot}

# 本小説の全話数
{all_chapter_number}

# 今回書いて欲しい話数
{chapter}

# 前話までの小説本文
{prev_novel}

----

この続きから記載してください

----
"""


# LangChainでキーワード抽出用プロンプトの設定
keyword_extraction_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", keyword_extraction_system_prompt),
        ("human", "{user_input}")
    ]
)


cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",cot_system_prompt),
        ("human", cot_human_prompt)
    ]
)

generate_story_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",generate_story_system_prompt),
        ("human", generate_story_human_prompt)
    ]
)

criticalA_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",criticalA_system_prompt),
        ("human", "{novel}")
    ]
)

criticalB_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",criticalB_system_prompt),
        ("human", "{novel}")
    ]
)

talk_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",taik_system_prompt),
        ("human", "{critical_sheet}")
    ]
)

next_generation_prot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",next_generation_prot_system_prompt),
        ("human", next_generation_prot_human_prompt)
    ]
)

next_generation_novel_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",next_generation_novel_system_prompt),
        ("human", next_generation_novel_human_prompt)
    ]
)

output_parser = StrOutputParser()

# なろうAPIを使って、ジャンルに基づいた評価の高い小説情報を取得する関数
def fetch_narou_novel_info(user_keyword):
    url = "https://api.syosetu.com/novelapi/api/"
    params = {
        "word": user_keyword,
        "order": "monthlypoint",
        "out": "json",
        "lim": 10
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        novels = response.json()[1:]
        return [{"title": novel.get("title", ""),
                 "summary": novel.get("story", ""),
                 "tag": novel.get("keyword", "").split()}
                for novel in novels]
    else:
        return ["検索した結果、関連小説情報を取得できませんでした。"]


def generate_novel_by_chapters(user_input, initial_novel_info, prompt):
    final_novel_output = ""
    past_content = ""
    chapter_num = 1

    while chapter_num <= all_chapter_number:

        chapter_input = {
            "all_chapter_number": all_chapter_number,
            "user_input": user_input,
            "prev_novel": past_content,
            "chapter": chapter_num,
            "information": initial_novel_info
        }

        sub_chain = prompt| model | output_parser
        current_chapter_output = sub_chain.invoke(chapter_input)

        final_novel_output += f"\n==== Chapter {chapter_num} ====\n" + current_chapter_output
        past_content += current_chapter_output

        chapter_num += 1

    return final_novel_output

def generate_final_novel_by_chapters(user_input, novel, critical_sheet, taik, next_novel_plot, prompt):
    final_novel_output = ""
    past_content = ""
    chapter_num = 1

    while chapter_num <= all_chapter_number:
        chapter_input = {
            "all_chapter_number": all_chapter_number,
            "user_input": user_input,
            "novel": novel,
            "critical_sheet": critical_sheet,
            "taik": taik,
            "next_novel_plot": next_novel_plot,
            "prev_novel": past_content,
            "chapter": chapter_num,
        }

        sub_chain = prompt| model | output_parser

        current_chapter_output = sub_chain.invoke(chapter_input)

        final_novel_output += f"\n==== Chapter {chapter_num} ====\n" + current_chapter_output
        past_content += current_chapter_output

        chapter_num += 1

    return final_novel_output

chain = (
    RunnableParallel(
        {
            "user_input": RunnablePassthrough(),
            "all_chapter_number": RunnablePassthrough(),
            "search_words": keyword_extraction_prompt | model | output_parser,
        }
    )
    .assign(naro_novel_info=RunnableLambda(lambda inputs: {"naro_novel_info": fetch_narou_novel_info(inputs["search_words"])}))
    .assign(
        cot_prompt_output=RunnableLambda(lambda inputs: {
            "cot_prompt_output": cot_prompt.invoke({"all_chapter_number":inputs["all_chapter_number"],"user_input": inputs["user_input"], "naro_novel_info": inputs["naro_novel_info"]})
        })
    )
    .assign(information=cot_prompt | model | output_parser)
    .assign(novel=RunnableLambda(lambda inputs: {
        "novel": generate_novel_by_chapters(inputs["user_input"], inputs["information"], generate_story_prompt)
    }))
    .assign(
        critical_sheet = RunnableParallel(
            {
            "criticalA_sheet":criticalA_prompt | model | output_parser,
            "criticalB_sheet":criticalB_prompt | model | output_parser
            }
        )
    )
    .assign(taik=talk_prompt | model | output_parser)
    .assign(next_novel_plot=next_generation_prot_prompt | model | output_parser)
    .assign(final_novel=RunnableLambda(lambda inputs: {
        "final_novel": generate_final_novel_by_chapters(
            inputs["user_input"],
            inputs["novel"],
            inputs["critical_sheet"],
            inputs["taik"],
            inputs["next_novel_plot"],
            next_generation_novel_prompt)
    }))
)

output = ""
output_user_input = ""
output_information = ""
output_novel = ""
output_criticalA_sheet = ""
output_criticalB_sheet = ""
output_taik = ""
output_next_novel_plot = ""
output_final_novel = ""
output_naro_novel_info = ""
output_search_words = ""
output_cot_prompt_output = ""

for chunk in chain.stream({"user_input":user_input, "all_chapter_number": str(all_chapter_number)}):
    if "user_input" in chunk:
        print(chunk["user_input"]["user_input"], end="", flush=True)
        output_user_input += chunk["user_input"]["user_input"]
    elif "search_words" in chunk:
        print(chunk["search_words"], end="", flush=True)
        output_search_words += chunk["search_words"]
    elif "naro_novel_info" in chunk:
        print(chunk["naro_novel_info"], end="", flush=True)
        output_naro_novel_info += str(chunk["naro_novel_info"])
    elif "cot_prompt_output" in chunk:
        if "cot_prompt_output" in chunk["cot_prompt_output"]:
            print(chunk["cot_prompt_output"]["cot_prompt_output"], end="", flush=True)
            output_cot_prompt_output += str(chunk["cot_prompt_output"]["cot_prompt_output"])
    elif "information" in chunk:
        print(chunk["information"],end="", flush=True)
        output_information += chunk["information"]
    elif "novel" in chunk:
        if "novel" in chunk["novel"]:
            print(chunk["novel"]["novel"], end="", flush=True)
            output_novel += chunk["novel"]["novel"]
    elif "critical_sheet" in chunk:
        if "criticalA_sheet" in chunk["critical_sheet"]:
            print(chunk["critical_sheet"]["criticalA_sheet"], end="", flush=True)
            output_criticalA_sheet += chunk["critical_sheet"]["criticalA_sheet"]
        elif "criticalB_sheet" in chunk["critical_sheet"]:
            print(chunk["critical_sheet"]["criticalB_sheet"], end="", flush=True)
            output_criticalB_sheet += chunk["critical_sheet"]["criticalB_sheet"]
    elif "taik" in chunk:
        print(chunk["taik"], end="", flush=True)
        output_taik += chunk["taik"]
    elif "next_novel_plot" in chunk:
        print(chunk["next_novel_plot"], end="", flush=True)
        output_next_novel_plot += chunk["next_novel_plot"]
    elif "final_novel" in chunk:
        if "final_novel" in chunk["final_novel"]:
            print(chunk["final_novel"]["final_novel"], end="", flush=True)
            output_final_novel += chunk["final_novel"]["final_novel"]


with open("output_sample06.txt", "w") as f:
    f.write("\n====user_user_input====\n")
    f.write(output_user_input)
    f.write("\n")
    f.write("\n====output_search_words====\n")
    f.write(output_search_words)
    f.write("\n")
    f.write("\n====output_naro_novel_info====\n")
    f.write(output_naro_novel_info)
    f.write("\n")
    f.write("\n====output_cot_prompt_output====\n")
    f.write(output_cot_prompt_output)
    f.write("\n")
    f.write("\n====output_information====\n")
    f.write(output_information)
    f.write("\n")
    f.write("\n====output_novel====\n")
    f.write(output_novel)
    f.write("\n")
    f.write("\n====output_critical_sheet====\n")
    f.write("\n====output_criticalA_sheet====\n")
    f.write(output_criticalA_sheet)
    f.write("\n")
    f.write("\n====output_criticalB_sheet====\n")
    f.write(output_criticalB_sheet)
    f.write("\n")
    f.write("\n====output_taik====\n")
    f.write(output_taik)
    f.write("\n")
    f.write("\n====output_next_novel_plot====\n")
    f.write(output_next_novel_plot)
    f.write("\n")
    f.write("\n====output_final_novel====\n")
    f.write(output_final_novel)
